- This notebook to get latent space of binary mixture dataset (log normalization) by using chemprop 

In [2]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [3]:
import pandas as pd
import numpy as np
from miscibility import normalize_concentrations_log
from chemprop_latent import chemprop_latent_space, build_mixture_latent_features_chemprop

In [4]:
df_mixture = pd.read_csv('df_mixture_20250502.csv')
# Log normalization
conc_cols = [f'conc{i}' for i in range(1, 3)]
df_mixture_norm = normalize_concentrations_log(df_mixture, conc_cols)
df_mixture_norm.head()

,smi1,conc1,smi2,conc2,miscibility
0,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCO,5.471754,1
1,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,[C@H]1([C@@H]([C@@H](OC([C@H]1O)C(=O)O)O[C@H]2...,3.819756,1
2,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...,4.116358,1
3,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)OS(=...,2.196230,1
4,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)O)O[...,1.993277,1


In [5]:
checkpoint_path='../src/models/chemprop/example_model_v2_regression_mol.ckpt'

In [6]:
x_latent, y = build_mixture_latent_features_chemprop(
    df=df_mixture_norm,
    smi_cols=["smi1", "smi2"],
    conc_cols=["conc1", "conc2"],
    target_col="miscibility",
    latent_fn=chemprop_latent_space,
    latent_fn_args={"checkpoint_path": checkpoint_path}
)

In [7]:
df_total = pd.concat([pd.DataFrame(x_latent),pd.DataFrame(y, columns=['miscibility'])], axis=1)
df_total.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,miscibility
0,-0.394030,-0.234446,-0.052132,0.728349,-0.138293,0.362742,0.432115,-0.326427,-1.116689,-0.391459,...,0.932251,0.920589,-0.271192,-1.084311,1.429382,0.683654,-1.242399,0.420842,0.119026,1
1,-0.403667,-0.141857,-0.155662,0.514603,-0.207111,0.424002,0.368113,0.002661,-0.978016,-0.290509,...,0.640568,0.841774,-0.238763,-0.816970,1.054817,0.644566,-0.957500,0.393161,0.049050,1
2,-0.519369,-0.046291,-0.323930,0.560490,-0.164046,0.457233,0.422266,-0.053809,-1.035232,-0.172511,...,0.598506,0.910867,-0.295952,-0.880982,1.187202,0.736888,-1.243009,0.391686,0.041209,1
3,-0.377324,-0.111257,-0.222519,0.463209,-0.183555,0.381745,0.350440,-0.047130,-0.793858,-0.192699,...,0.520744,0.634037,-0.187247,-0.708163,0.955804,0.541579,-0.927625,0.375801,0.036138,1
4,-0.372274,-0.083259,-0.219775,0.450107,-0.172099,0.364624,0.333705,-0.050856,-0.780366,-0.166237,...,0.507472,0.641236,-0.202588,-0.684405,0.943059,0.537758,-0.929753,0.342465,0.039941,1


In [8]:
df_total.to_csv('df_total_binary_chemprop_log_20250930.csv', index=False)